In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import itertools
import sklearn
import scipy
import seaborn as sns
from matplotlib import pyplot as plt
import squarify
import matplotlib.ticker as ticker
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy.spatial.distance import euclidean
import sys
from sklearn.preprocessing import MinMaxScaler
import math

In [ ]:
df = pd.read_csv('../input/crimes-in-boston/crime.csv',encoding='latin-1')

df.drop("INCIDENT_NUMBER",axis=1, inplace=True) 

df[["DATE","TIME"]]=df['OCCURRED_ON_DATE'].str.split(" ",expand=True) 

In [ ]:

df.describe()

In [ ]:
df.info()

In [ ]:
# plot line chart
def lineplt(x,y,xlabel,ylabel,title,size,tick_spacing):
    fig,ax=plt.subplots(figsize = size)
    plt.plot(x,y)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
    plt.xlabel(xlabel,fontsize = 15)
    plt.ylabel(ylabel,fontsize = 15)
    plt.title(title,fontsize = 20)
    plt.show()

# Create 2 columes DateFrame
def createdf(c1,d1,c2,d2):
    dic = {c1:d1,c2:d2}
    df = pd.DataFrame(dic)
    return df

# Plot histogram
def plthis(d,bin, title):
    plt.figure(figsize=(10,8))
    plt.hist(d, bins=bin)
    plt.title(title, fontsize = 20)
    plt.show()

In [ ]:
# Put Date and Count into a new Dataframe
c = createdf("Date",df["DATE"].value_counts().index,"Count",df["DATE"].value_counts())

# c is the total number of crimes per day
c.head(5)

## 2.1 The distribution of the Counts of Crimes by date

In [ ]:
plthis(c["Count"],50, "Crimes Count Distribution")

In [ ]:
print('skewness is ' + str(c['Count'].skew()))
print('kurtosis is ' + str(c['Count'].kurt()))

In [ ]:
bin=pd.cut(c["Count"],50)
fre= createdf("Bin",bin.value_counts().index,"Count",bin.value_counts())
fre_sort = fre.sort_values(by = "Bin", ascending = True)

### 2.1.1 Shapiro-Wilk test
(For N > 5000 the W test statistic is accurate but the p-value may not be.)

In [ ]:
(_,p) = scipy.stats.shapiro(fre_sort["Count"])
print('p-value is ' + str(p))

### 2.1.2 Kolmogorov-Smirnov test

In [ ]:
(_,p) = scipy.stats.kstest(fre_sort["Count"],'norm')
print('p-value is ' + str(p))

From the result of this two tests, we can see that the p value is very small, much smaller than 5%. So we can conclude that the distribution is **Significantly different** from normal distribution under 95% confidence.

## 2.2 Distribution of Crimes by Time

In [ ]:
c=c.sort_values(by="Date",ascending = True)
lineplt(c["Date"],c["Count"],"Date","Count","Crimes by Time",(20,15),80)

From the chart above, we can see there are many peaks and troughs and they shows a kind of pattern like "sin" function. I'm going to use some numbers and charts to describe the pattern in detail.

In [ ]:
fig = plt.figure(figsize=(16,16))
ax1 = fig.add_subplot(411)
fig = plot_acf(c["Count"],lags=200,ax=ax1)
plt.title('Autocorrelation Lag=200')
ax2 = fig.add_subplot(412)
fig = plot_pacf(c["Count"],lags=200,ax=ax2)
plt.title('Partial Autocorrelation Lag=200')
ax3 = fig.add_subplot(413)
fig = plot_acf(c["Count"],lags=15,ax=ax3)
plt.title('Autocorrelation Lag=15')
ax4 = fig.add_subplot(414)
fig = plot_pacf(c["Count"],lags=15,ax=ax4)
plt.title('Partial Autocorrelation Lag=15')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                wspace=None, hspace=0.5)
plt.show()


By looking at the Autocorrelation and Partical Autocorrelation (lag = 200 and lag = 15), we can conclude that:

(1) From lag 1 to lag 100, the correlation is positive and from lag 100 to lag 200, the correlation is negative. So around every 100 days, the correlation will be reversed. This can describe the "Sin" shape.

(2) When we make the lag shorter, we can see more details about the correlation. The partical correlations are significant when lag 1, lag 6 and lag 7. So we can conclude that the crimes are correlated with yesterday and the same day in last week.

In [ ]:
res = sm.tsa.seasonal_decompose(c['Count'],freq=12,model="additive")
# # original = res
trend = res.trend
seasonal = res.seasonal
residual = res.resid

fig,ax=plt.subplots(figsize = (20,15))
ax1 = fig.add_subplot(411)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(80))
ax1.plot(c['Count'], label='Original')
ax1.legend(loc='best')
ax2 = fig.add_subplot(412)
ax2.xaxis.set_major_locator(ticker.MultipleLocator(80))
ax2.plot(trend, label='Trend')
ax2.legend(loc='best')
ax3 = fig.add_subplot(413)
ax3.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax3.plot(seasonal[:100],label='Seasonality')
ax3.legend(loc='best')
ax4 = fig.add_subplot(414)
ax4.xaxis.set_major_locator(ticker.MultipleLocator(80))
ax4.plot(residual, label='Residuals')
ax4.legend(loc='best')
plt.tight_layout()


After seasonal decomposing, we can have a clear view of the pattern of the distribution of crimes. But there is one problem. When we try to plot the data, the chart will have different shape if we use different scales. For example, if the range of y axis is between 0~300, then the variation will be very clear, and we can see if there is a trend. But if the range is between 0~3000, then the variation will be unclear and the shape of the date could be a straight line. So we need to value the stationarity of the data becasue I'm planning to use ARIMA model.

# 2.2.1 ADF Test

In [ ]:
def test_stationarity(series,mlag = 365, lag = None,):
    print('ADF Test Result')
    res = adfuller(series, maxlag = mlag, autolag = lag)
    output = pd.Series(res[0:4],index = ['Test Statistic', 'p value', 'used lag', 'Number of observations used'])
    for key, value in res[4].items():
        output['Critical Value ' + key] = value
    print(output)

In [ ]:
test_stationarity(c['Count'],lag = 'AIC')

The report shows that, the "used lag" is 34 and p value is 0.19. So we can conclude, in the range of 34 days, we can **NOT Reject** the null hypotheses which is the time series is non-stationary.

## 2.3 ARIMA Model

Because the data is not stationary, we need to do first difference to the date in order to make it stationary.

In [ ]:
d1 = c.copy()
d1['Count'] = d1['Count'].diff(1)
d1 = d1.dropna()
lineplt(d1["Date"],d1["Count"],"Date","Count","Crimes by Time",(20,15),80)
print('Average= '+str(d1['Count'].mean()))
print('Std= ' + str(d1['Count'].std()))
print('SE= ' + str(d1['Count'].std()/math.sqrt(len(d1))))
print(test_stationarity(d1['Count'],lag = 'AIC'))

After the fist differencing, the chat looks much more stationary and the ADF test shows a pretty low p value. So we can rejct H0. The Time series is stational after first differencing.

In [ ]:
fig_2 = plt.figure(figsize=(16,8))
ax1_2 = fig_2.add_subplot(211)
fig_2 = plot_acf(d1["Count"],lags=15,ax=ax1_2)
ax2_2 = fig_2.add_subplot(212)
fig_2 = plot_pacf(d1["Count"],lags=15,ax=ax2_2)

The Autocorrelation and Partial Autocorrelation charts are not very perfet. We can see there is a seasonal pattern every 7 days. 

We will deal with it later. But let's build the ARIMA model first

In [ ]:
timeseries = c['Count']
p,d,q = (4,1,2)
arma_mod = ARMA(timeseries,(p,d,q)).fit()
summary = (arma_mod.summary2(alpha=.05, float_format="%.8f"))
print(summary)

In [ ]:
predict_data = arma_mod.predict(start='2016-07-01', end='2017-07-01', dynamic = False)
timeseries.index = pd.DatetimeIndex(timeseries.index)
fig, ax = plt.subplots(figsize=(20, 15))
ax = timeseries.plot(ax=ax)
predict_data.plot(ax=ax)
plt.show()